# ctypes: C library call python function 


* Change the current working directory into `./demo`

In [2]:
%cd demo

J:\SEU\SEE\PySEE\home\notebook\demo


In [2]:
%pwd

'J:\\SEU\\SEE\\PySEE\\home\\notebook\\demo'

## 1 Bisection Methon Shared Library in C

* using `__cdecl`calling convention

$y=f(x)$

In [3]:
%%file ./include/roots.h
#ifndef ROOTS_H
#define ROOTS_H

#include <math.h>
// function definition
typedef double (*fun)(double);

int rtbis(fun func,double y,double x1, double x2, double xacc,double *rtb);
/*
	The program uses the bisection method to solve the equation
		f(x)-y = 0.
	The solution is to be in [x1,x2] and it is assumed that
		(f(x1)-y)*(f(x2)-y) <= 0.
	The solution is returned in rtb, and it is to be in error by at most xacc.
	
	return value is an error indicator.
	  If =0, the solution has been computed satisfactorily.
	  If =1, (f(x1)-y)*(f(x2)-y) was greater than 0, contrary to assumption 
      If =2, exceeded the maximum number of iteration 
*/
#endif

Overwriting ./include/roots.h


### The ways to pass arguments to the function


Following are the two ways to **pass arguments** to the function:

* Pass by value
* Pass by reference

#### Pass-by-Value

In pass-by-value, a **copy** of argument is created and passed into the function. The invoked function works on the "clone", and **cannot modify the original copy**.

In C, fundamental types (such as int and double) are passed by value. That is, you cannot modify caller's value inside the function

####  Pass-by-Reference

In pass-by-reference, a **reference** of the caller's variable is passed into the function. In other words, the invoked function works on the **same** data. 

* If the invoked function modifies the parameter, the same caller's copy will be modified as well.



In [4]:
%%file ./src/roots.c
/*
  Numerical Recipes http://numerical.recipes
*/ 
#include <math.h>
#include "roots.h"

int rtbis(fun func,double y,double x1, double x2, double xacc,double *rtb)
{
 /*
	The program uses the bisection method to solve the equation
		f(x)-y = 0.
	The solution is to be in [x1,x2] and it is assumed that
		(f(x1)-y)*(f(x2)-y) <= 0.
	The solution is returned in rtb, and it is to be in error by at most xacc.
	
	return value is an error indicator.
	  If =0, the solution has been computed satisfactorily.
	  If =1, (f(x1)-y)*(f(x2)-y) was greater than 0, contrary to assumption 
      If =2, exceeded the maximum number of iteration 
*/
	const int IMAX=100; // the maximum number of iteration
    float dx,f,fmid,xmid;

	f=(*func)(x1)-y;
	fmid=(*func)(x2)-y;
	if (f*fmid >= 0.0) // endpoints do not straddle y=0
       return 1; 
    // init the root value: rtb
	*rtb = f < 0.0 ? (dx=x2-x1,x1) : (dx=x1-x2,x2);
	for (int i=1;i<=IMAX;i++) {
		fmid=(*func)(xmid=(*rtb)+(dx *= 0.5))-y;
		if (fmid <= 0.0) *rtb=xmid;
		if (fabs(dx) < xacc || fmid == 0.0) 
           return 0;
  	}
    // Exceeded the maximum number of iteration
    return 2;
}

Overwriting ./src/roots.c


**Windows**

In [5]:
!gcc -c -O3 -Wall -fPIC -o ./obj/roots.o  ./src/roots.c -I./include
!gcc -shared -o ./bin/libroots.dll  ./obj/roots.o

## 2 Caller in C

$f(x)=x^2$

$4=f(x)$

```
int rtbis(fun func,double y,double x1, double x2, double xacc,double *rtb)

double xl, xu, epsilon, root;
ier=rtbis(fcn,y,xl, xu, epsilon, &root);
```
*  &root > double *rtb

In [8]:
%%file ./src/rtbisApp.c

#include <stdio.h>
#include <string.h>
#include <stdlib.h>
#include "roots.h"

double fcn(double x)
{
	double result;
 main()
{
	double xl, xu, epsilon, root;
	int ier;
	xl = 0.1;
	xu = 3.2;
	epsilon = 0.001;
    double y=4;
	// Calculate root
	ier=rtbis(fcn,y,xl, xu, epsilon, &root);
	// Print answers
	printf("root = %14.7e  ier = %1d", root, ier);
	return 0;
}	result = x * x;
	return result;
}

int

Overwriting ./src/rtbisApp.c


In [9]:
!gcc -o  ./bin/rtbisApp ./src/rtbisApp.c -L./bin/ -lroots -I./include

In [10]:
!.\bin\rtbisApp 

root = 1.9996581e+000  ier = 0


## 3 Python API 


**CDLL**

* Instances of this class represent loaded shared libraries. Functions in these libraries use the standard `C calling convention`, and are assumed to return int.

```
int rtbis(fun func,double y,double x1, double x2, double xacc,double *rtb)

rtbis.argtypes = [fun,c_double, c_double,c_double,c_double,POINTER(c_double)]
rtbis.restype = c_int
```

* POINTER(c_double) > double *rtb

In [11]:
from ctypes import * 

rtapi = CDLL('.\\bin\\libroots.dll')
fun = CFUNCTYPE(c_double, c_double)

def biroot(fn,y,xl,xu,eps): 
    rtbis = rtapi.rtbis
    rtbis.argtypes = [fun,c_double, c_double,c_double,c_double,POINTER(c_double)]
    rtbis.restype = c_int

    rtb=c_double()
    ier=rtbis(fn,y,xl,xu,eps,byref(rtb))
    return rtb.value,ier

In [12]:
def fn(x):        
    return x**2

_fn = fun(fn)
xl = c_double(0.2) 
xu = c_double(3)
eps=0.001
y=4

rtb,ier=biroot(_fn,y,xl,xu,eps)
print(rtb,ier)

1.999902367591858 0


##  Reference

* [Numerical Recipes](http://numerical.recipes)

* [GSL - GNU Scientific Library](https://www.gnu.org/software/gsl/)

* [Intel Math Kernel Library](https://software.intel.com/en-us/mkl)